In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/Users/juliasuter/Documents/repositories/asf_core_data")
import os

from asf_core_data import generate_and_save_mcs
from asf_core_data import load_preprocessed_epc_data
from asf_core_data.getters.epc import data_batches
from asf_core_data.pipeline.preprocessing import preprocess_epc_data
from asf_core_data.pipeline.data_joining import merge_install_dates, merge_proc_datasets


2023-02-24 17:30:12,668 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [13]:
from asf_core_data.config import base_config
base_config.MCS_EPC_CHARACTERISTIC_FIELDS

['UPRN']

In [2]:
LOCAL_DATA_DIR = '/Users/juliasuter/Documents/ASF_data'

In [3]:
# Check whether newest batch shows up a newest in local data dir
print("Local input dir\n---------------")
print("Available batches:", data_batches.get_all_batch_names(data_path=LOCAL_DATA_DIR, check_folder='inputs'))
print("Newest batch:", data_batches.get_most_recent_batch(data_path=LOCAL_DATA_DIR))

Local input dir
---------------
Available batches: ['2021_Q2_0721', '2022_Q1_complete', '2021_Q4_0721', '2020_Q3_1220', '2022_Q3_complete']
Newest batch: 2022_Q3_complete


In [4]:
# Process new batch of EPC data
epc_full = preprocess_epc_data.load_and_preprocess_epc_data(
    data_path=LOCAL_DATA_DIR, batch="newest", subset='GB',
    reload_raw=True
)

KeyboardInterrupt: 

In [20]:
# Get MCS and join with MCS
generate_and_save_mcs()

inputs/MCS/latest_raw_data/mcs_installations_2021.xlsx
inputs/MCS/latest_raw_data/mcs_installations_2022_q1.xlsx
inputs/MCS/latest_raw_data/mcs_installations_2022_q2.xlsx
inputs/MCS/latest_raw_data/mcs_installations_2022_q3.xlsx


/Users/juliasuter/Documents/repositories/asf_core_data/asf_core_data/pipeline/mcs/generate_mcs_data.py:132: UserWarning: Not all installation file columns are the same.
  warnings.warn("Not all installation file columns are the same.")


Number of records before removing duplicates: 170275
Number of records after removing duplicates: 170237
Shape of loaded data: (168574, 31)


/Users/juliasuter/Documents/repositories/asf_core_data/asf_core_data/pipeline/mcs/process/process_mcs_installations.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hps["cluster"].loc[


Saved in S3: /outputs/MCS/mcs_installations_230224.csv
Getting EPC data...
Forming a matching...
- Forming an index...
- Forming a comparison...
- Computing a matching...
Joining the data...
After joining:
-----------------
Total records: 267336
Number matched with EPC: 238510


Saved in S3: /outputs/MCS/mcs_installations_epc_full_230224.csv


/Users/juliasuter/Documents/repositories/asf_core_data/asf_core_data/pipeline/mcs/process/mcs_epc_joining.py:385: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df["last_epc_before_mcs"].iloc[last_epc_before_mcs_indices] = True


Saved in S3: /outputs/MCS/mcs_installations_epc_most_relevant_230224.csv


In [22]:
# Load the processed EPC data 
prep_epc = load_preprocessed_epc_data(data_path=LOCAL_DATA_DIR, version='preprocessed',
                                       #usecols=['UPRN', 'INSPECTION_DATE', 'HP_INSTALLED', 'HP_TYPE'], 
                                       batch='newest'
                                    )


In [ ]:
# Add more precise estimations for heat pump installation dates via MCS data
epc_with_MCS_dates = merge_install_dates.manage_hp_install_dates(
    prep_epc
)

epc_with_MCS_dates.shape

In [ ]:
epc_mcs_processed = merge_proc_datasets.merge_proc_epc_and_mcs_installs(epc_with_MCS_dates)